In [ ]:
from PosSelect_Functions_Old import *
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
import pandas as pd
import numpy as np
import copy
import seaborn as sns
from scipy.stats import mannwhitneyu as mwu
from scipy.stats import ttest_ind
from scipy.stats import ttest_rel
from statsmodels.stats.multitest import fdrcorrection
from scipy.stats import wilcoxon
from scipy.optimize import curve_fit
from scipy.stats import fisher_exact
from scipy.stats import norm
from collections import Counter

hfont = {'fontname':'Arial'}
plt.rcParams["font.family"] = "Arial"

#Code borrowed heavily from here: https://stackoverflow.com/questions/62375034/find-non-overlapping-area-between-two-kde-plots
plt.rcParams.update(
    {"text.usetex": False}
)

def parse_table(i):
    return [int(x) for x in i.replace("[", "").replace("]", "").split(",")]

d_abrev = {"LiangSteinNeuron":"FC exc. neur.", "FetalChondrocytes":"F chond.", "SertoliMale":"FG sertoli", "preGC_IIaFemale":"FG preGC IIa",\
          "NeuralFemale":"FG neur.", "FetalGonadImmuneFemale":"FG immune", "VIP":"AC VIP inh. neur.", "LiangSteinProgenitor":"FC prog.",\
          "AdultHeartVentricularCardiomyocyte":"AH cardiomyo.", "AdultLoopOfHenle":"AK loop of henle", "FetalBrainNeurGlioblast_CB_VZ":"FCB glioblast",\
         "AdultProximalTubule":"AK prox. tub.", "FetalLeydigMale":"FG leydig", "SST":"AC SST inh neur.", "KosoyRoussosControlMicroglia":"AC microglia",\
         "FetalBrainFloorPlate":"FB fl. plate", "FetalArterialECs":"FH endoth.", "ASCT":"AC astro.", "FetalBrainCOP":"FB COP",\
         "AMY":"AA neur.", "PVALB":"AC PVALB inh neur.", "ITL23":"AC L2-3 IT neur.", "FetalBrainNeurCB_GNP_IPC_1":"FB inter. prog.", "FetalBrainNeurDAergic":"FB DA neur.",\
          "OGC":"AC Oligo.", "D1Pu":"AP D1 inh neur.", "FetalBrainNeurSerotonergic":"FB 5-HT neur.", "FetalBrainNeurDRG_2":"FS DRG neur.",\
          "FetalHeartPericytes":"FH peri.", "FetalHeartEndocardium":"FH endocard.", "FetalHeartCardiacFibroblasts":"FH fibro.", "FetalBrainNeurPurkinje_6":"FCB Purk. inh neur.",\
          "AdultHeartSmoothMuscle":"AH smooth musc.", "FetalBrainRoofPlate":"FB ro. plate"}

In [ ]:
#Plotting in and out of Alu elements for chondrocyte ZNF460
v = pd.read_csv("MA1596.1_ZNF460_400_FetalChondrocytes_Fixed.csv")
v = v[(v["1"] > 500) | ((v["2"] > 500) & (v["4"] > 500))]
vv = pd.read_csv("MA1596.1_ZNF460_400_FetalChondrocytes_Poly.csv")


vv_ref = vv[vv["Human ref"] == vv["Chimp ref"]]
vv_alt = vv[vv["Human alt"] == vv["Chimp ref"]]
vv_ref["fixed logfc"] = -vv_ref["logfc"].astype(float)
vv_alt["fixed logfc"] = vv_alt["logfc"].astype(float)
vv = pd.concat([vv_ref, vv_alt])
vv = vv[(vv["1"] > 500) | ((vv["2"] > 500) & (vv["3"] > 500))]
vv = vv[vv["Human ref"] != vv["Chimp ref"]]

v2 = v.copy()
vv2 = vv.copy()

cuttt = 0.6

z = list(vv2["abs logfc"])
z.sort()
cutoff = z[int(floor((len(z)*cuttt)))]

vvv = prepare_alpha(v2, vv2, stat = "abs logfc")
#vvv["PhyloP"] = -vvv["PhyloP"]
alpha = compute_alpha_cutoff(vvv, plot = True, window = [0, 0.4], cutoff = cutoff, title = "ZNF460 binding sites in fetal chondrocytes")
plt.xlabel("Predicted absolute log$_{2}$ fold-change in CA", fontsize = 20)
print(alpha)
print((fisher_exact(alpha[-2], alternative = "greater")[1] + fisher_exact(alpha[-3], alternative = "greater")[1])/2)

In [ ]:
rep_f = pd.read_csv("Pseudos_Repeats/HumChp_NC_Final_CREs_NoHLA_rmsk.txt", sep = "\t", header = None)
rep_p = pd.read_csv("Pseudos_Repeats/HumPoly_NC_Final_CREs_NoHLA_rmsk.txt", sep = "\t", header = None)


rep_f.columns = [0, "Repeat"]
rep_p.columns = [0, "Repeat"]

In [ ]:
v2 = v2.set_index("Position.1").join(rep_f.set_index(0))


In [ ]:
vv2 = vv2.set_index("Position").join(rep_p.set_index(0))
vv2

In [ ]:
from collections import Counter

reps = Counter(list(v2["Repeat"]) + list(vv2["Repeat"]))
alu = []
for key in reps.keys():
    if key.startswith("Alu") or key.startswith("ALU"):
        alu.append(key)

In [ ]:
vx = v2[~v2["Repeat"].isin(alu)]
vvx = vv2[~vv2["Repeat"].isin(alu)]

cuttt = 0.6

z = list(vvx["abs logfc"])
z.sort()
cutoff = z[int(floor((len(z)*cuttt)))]

vvv = prepare_alpha(vx, vvx, stat = "abs logfc")
#vvv["PhyloP"] = -vvv["PhyloP"]
alpha = compute_alpha_cutoff(vvv, plot = True, window = [0, 0.4], cutoff = cutoff, title = "ZNF460 binding sites in Alus in fetal chondrocytes")
plt.xlabel("Predicted absolute log$_{2}$ fold-change in CA", fontsize = 20)
print(alpha)
print((fisher_exact(alpha[-2], alternative = "greater")[1] + fisher_exact(alpha[-3], alternative = "greater")[1])/2)

In [ ]:
vx = v2[~v2["Repeat"].isin(alu)]
vvx = vv2[~vv2["Repeat"].isin(alu)]

cuttt = 0.6

z = list(vvx["abs logfc"])
z.sort()
cutoff = z[int(floor((len(z)*cuttt)))]

vvv = prepare_alpha(vx, vvx, stat = "abs logfc")
#vvv["PhyloP"] = -vvv["PhyloP"]
alpha = compute_alpha_cutoff(vvv, plot = True, window = [0, 0.4], cutoff = cutoff, title = "ZNF460 binding sites not in Alus in fetal chondrocytes")
plt.xlabel("Predicted absolute log fold-change in CA")
print(alpha)
print((fisher_exact(alpha[-2], alternative = "greater")[1] + fisher_exact(alpha[-3], alternative = "greater")[1])/2)

In [ ]:
#Read in the data
v = pd.read_csv("Fixed_FetalChondrocytes.txt.gz", sep = "\t")
vv = pd.read_csv("Poly_MAF0.25_FetalChondrocytes.txt.gz", sep = "\t")


try:
    vv = add_unfold(vv)
    vv_ref = vv[vv["Human ref"] == vv["Chimp ref"]]
    vv_alt = vv[vv["Human alt"] == vv["Chimp ref"]]
    vv_ref["fixed logfc"] = -vv_ref["logfc"].astype(float)
    vv_alt["fixed logfc"] = vv_alt["logfc"].astype(float)
    vv = pd.concat([vv_ref, vv_alt])
except:
    pass

te_blacklist = pd.read_csv("BlacklistTE_Variants.txt", sep = "\t")
    
v = v[~v["Position"].isin(te_blacklist["Position"])]
vv = vv[~vv["Position"].isin(te_blacklist["Position"])]



In [ ]:
v.index = v["Position"]
v = v.join(rep_f.set_index(0)).drop_duplicates("Position").dropna()

vv.index = vv["Position"]
vv = vv.join(rep_p.set_index(0)).drop_duplicates("Position").dropna()

from collections import Counter

reps = Counter(list(v["Repeat"]) + list(vv["Repeat"]))
alu = []
for key in reps.keys():
    if key.startswith("Alu") or key.startswith("ALU"):
        alu.append(key)

va = v[v["Repeat"].isin(alu)]
vva = vv[vv["Repeat"].isin(alu)]
vna = v[~v["Repeat"].isin(alu)]
vvna = vv[~vv["Repeat"].isin(alu)]

In [ ]:
import os

dff = pd.DataFrame()
files = os.listdir("Alu")
for file in files:
    if file.endswith("_Alu.txt"):
        df = pd.read_csv("Alu/" + file, sep = "\t")
        df = df[df["ID"].isin(["All alu", "All non-alu", "All non-alu, Rep240"])]
        df = df[df["Proportion"] == 0.95]
        df = df[df["Metric"] == "abs logfc"]
        df["Cell type"] = np.repeat(d_abrev[file.replace("_Alu.txt", "")], df.shape[0])
        dff = pd.concat([dff, df])
dff.sort_values("alpha").tail(50)

In [ ]:
#Create the R code
df2 = dff[dff["ID"] == "All alu"]
for index, row in df2.iterrows():
    print(row["Cell type"], "fisher.test(matrix(c(" + row["[[dc2, du2], [pc2, pu2]]"].replace("[[", "").replace("]]", "").replace("], [", ",") + "), nrow = 2))")

In [ ]:
#Renaming things
d_cat = {"All alu":"All sites in Alus", "All non-alu":"All sites not in Alus", "All non-alu, Rep240":"All post-primate sites in non-Alu repeats"}

In [ ]:
#Confidence intervals to plot
d_CI = {"AC VIP inh. neur.": [0.9976683, 1.0359399], "AC SST inh neur.": [0.9912361, 1.0292874], "FG sertoli":[1.029064, 1.068352], \
       "AC PVALB inh neur.": [0.9943275, 1.0324658], "FG preGC IIa":[1.023174, 1.062298], "AC Oligo.":[0.994174,  1.032304], \
       "FG neur.":[1.006386, 1.044910], "FC prog.":[1.019341, 1.058331], "AH smooth musc.":[0.9925567, 1.0306218], "AH cardiomyo.":[1.027731, 1.066996], \
       "AK loop of henle":[1.027731, 1.066996], "AK prox. tub.":[0.9891001, 1.0270509], "AA neur.":[0.9930829, 1.0311851], "AC astro.":[0.985658, 1.023498], \
       "AP D1 inh neur.":[0.9375516, 0.9737225], "FH endoth.":[0.9865731, 1.0244648], "FB COP":[0.9927941, 1.0308773], "FB fl. plate":[1.007707, 1.046339], \
       "FB inter. prog.":[1.016047, 1.054903], "FB DA neur.":[1.027731, 1.066996], "FS DRG neur.":[0.9782636, 1.0158647], "FCB glioblast":[1.020377, 1.059427], \
       "FCB Purk. inh neur.":[0.9816272, 1.0193546], "FB 5-HT neur.":[0.9991928, 1.0374820], "FB ro. plate":[1.031028, 1.070456], "F chond.":[1.084564, 1.125781], \
       "FG immune":[0.9645273, 1.0016309], "FH fibro.":[1.012894, 1.051685], "FH endocard.":[0.9992122, 1.0375505], "FH peri.":[1.016930, 1.055841], "FG leydig":[1.064013, 1.104568], \
       "AC L2-3 IT neur.":[0.991545, 1.029579], "AC microglia":[0.9804507, 1.0181336], "FC exc. neur.":[0.9839536, 1.0217614]}
out = []
for key in d_CI.keys():
    df2 = dff[dff["Cell type"] == key]
    for index, row in df2.iterrows():
        t = parse_table(row["[[dc2, du2], [pc2, pu2]]"])
        out.append([key, fisher_exact([t[0:2], t[2:]])[0], d_cat[row["ID"]]])
dfp = pd.DataFrame(out)
dfp.columns = ["Cell type", "Odds ratio", "Category"]

In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300

palette = {"All sites in Alus":"#F5C9471A", "All sites not in Alus":"#47F5C91A", "All post-primate sites in non-Alu repeats":"#735D751A"}

fig, ax = plt.subplots(figsize = (18, 6))
sns.barplot(data = dfp, x = "Cell type", hue = "Category", y = "Odds ratio", palette = palette)
plt.xticks(rotation = 90, size = 16)
plt.yticks(size = 16)

plt.ylim(0.9, 1.15)
plt.title("Stronger evidence for positive selection in Alu elements in most cell types", size = 20)
plt.ylabel("Odds ratio", size = 18)
plt.xlabel("Cell type", size = 18)
plt.legend(fontsize = 16)
i = -0.265


for key in d_CI.keys():
    val = float(dfp[(dfp["Cell type"] == key) & (dfp["Category"] == "All sites in Alus")]["Odds ratio"])
    plt.vlines(i, d_CI[key][0], val, color='black')
    plt.hlines(d_CI[key][0], i-0.075, i+0.075, color='black')

    plt.vlines(i, val, d_CI[key][1], color='black')
    plt.hlines(d_CI[key][1], i-0.075, i+0.075, color='black')
    
    #print(i+0.05 - i - 0.05)
    i += 1